## Scrapes Volksläufe from Runner's World Website

Scrapes information from every Volkslauf listed on https://www.runnersworld.de/volkslaeufe-strassenlaeufe/

In [1]:
from autoscraper import AutoScraper

In [2]:
ex1 = (
    "https://www.runnersworld.de/laufkalender/badischemeilekarlsruhe/",
    [
        "Badische Meile Karlsruhe",
        # "Stadtlauf / Citylauf", "Volkslauf / Straßenlauf",
        "08.05.2022",
        "76136 Karlsruhe, Hermann-Veit-Straße 3", 
        "www.badischemeile.de",
        "info@badischemeile.de"
    ]
)

ex2 = (
    "https://www.runnersworld.de/laufkalender/placentia-half-marathon/",
    [
        "Placentia Half Marathon"
        # "Stadtlauf / Citylauf", "Volkslauf / Straßenlauf",
        "08.05.2022",
        "03.05.2022",
        "IT 29121 Piacenza, Palazzo Farnese",
        "www.placentiahalfmarathon.org",
        "info@placentiahalfmarathon.it"
    ]
)

ex3 = (
    "https://www.runnersworld.de/laufkalender/wings-for-life-world-run-muenchen/",
    [
        "Wings for Life World Run München",
        "Volkslauf / Straßenlauf",
        "Landschaftslauf",
        "Spendenlauf / Charity-Lauf",
        "08.05.2022",
        "80809 München, Hans-Jochen-Vogel-Platz (Olympiapark/Nord)",
        "www.wingsforlifeworldrun.com/de/locations/munich"
    ]
)

ex4 = (
    "https://www.runnersworld.de/laufkalender/lisbon-eco-marathon/",
    [
        "Lisbon Eco Marathon: abgesagt",
        "Stadtlauf / Citylauf", "Volkslauf / Straßenlauf",
        "08.05.2022",
        "PT 1070-051 Lissabon, Parque Eduardo VII de Inglaterra",
        "maratonadelisboa.pt",
        "lisbonecomarathon@clubechronos.com"
    ]
)

data_train = [
    ex1,
    ex2, 
    ex3,
    ex4
]

In [3]:
# training
scraper = AutoScraper()
for url, wanted_list in data_train:
    scraper.build(url, wanted_list, update=True)

In [56]:
# create a scraper for the tables...autoscraper can't handle varying tables
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd

# url = "https://www.runnersworld.de/laufkalender/gutenbergmarathonmainz/"
# url = "https://www.runnersworld.de/laufkalender/lissabonhalbmarathon/"
# url = "https://www.runnersworld.de/laufkalender/wings-for-life-world-run-muenchen/"
url = "https://www.runnersworld.de/laufkalender/schweinelauf-wuelfrath/"
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html)
table = soup.find('table')

list_rows = [] #make list for rows
for row in table.findAll('tr'):
    list_cells = [] # array for row
    for cell in row.findAll(['td', 'th']):
        text = cell.text.replace('&nbsp;', '') #replace nonbreaking space
        
        # clean
        text = text.replace('ü', 'ue')
        text = text.replace(' Euro', '')
        text = text.replace(' km', '')
        text = text.replace(' Uhr', '')
        text = text.replace(',', '.')
        text = text.lower()

        list_cells.append(text)
    list_rows.append(list_cells)
    
df = pd.DataFrame(list_rows[1:], columns=list_rows[0])
df

,distanz,startzeit,startgebuehr
0,12.1,11.10,15.00 - 23.00
1,5.3,11.00,15.00 - 23.00


In [57]:
# clean df
def split_startgebuehr(x: pd.Series) -> pd.DataFrame:
    x = x.str.split(' - ').to_list()
    return pd.DataFrame(x, columns=['startgebuehr_min', 'startgebuehr_max'])

def append_minmax_startgebuehr(df, df_startgebuehr):
    df['startgebuehr_min'] = df_startgebuehr['startgebuehr_min']
    df['startgebuehr_max'] = df_startgebuehr['startgebuehr_max']
    df.drop('startgebuehr', axis=1, inplace=True) 
    return df

df_startgebuehr = split_startgebuehr(df['startgebuehr'])
df = append_minmax_startgebuehr(df, df_startgebuehr)
df 

,distanz,startzeit,startgebuehr_min,startgebuehr_max
0,12.1,11.10,15.00,23.00
1,5.3,11.00,15.00,23.00


In [67]:
df['startgebuehr_min'] = df['startgebuehr_min'].astype(float)
df['startgebuehr_max'] = df['startgebuehr_max'].astype(float)
df['distanz'] = df['distanz'].astype(float)
df['startzeit'] = df['startzeit'].str.replace('.', ':', regex=False).astype(str)

print(df)
print()
df.info()

   distanz startzeit  startgebuehr_min  startgebuehr_max
0     12.1     11:10              15.0              23.0
1      5.3     11:00              15.0              23.0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   distanz           2 non-null      float64
 1   startzeit         2 non-null      object 
 2   startgebuehr_min  2 non-null      float64
 3   startgebuehr_max  2 non-null      float64
dtypes: float64(3), object(1)
memory usage: 192.0+ bytes


In [95]:
def scrape_table(url: str) -> list:
    """
    Extracts the table from a runnersworld.de table, containing information
    about the run. Returns a list of lists. Each inner list corresponds
    to a row in the table. E.g a table with 2 rows and 3 cols will yield:
    X = [[x11, x12, x13],
         [x21, x21, x22]]
    """
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html)
    table = soup.find('table')

    list_rows = [] #make list for rows
    for row in table.findAll('tr')[1:]:
        list_cells = [] #array for row
        for cell in row.findAll('td'):
            text = cell.text.replace('&nbsp;', '') #replace nonbreaking space
            list_cells.append(text)
        list_rows.append(list_cells)
        
    return list_rows

In [82]:
# extract the links of the runs from https://www.runnersworld.de/volkslaeufe-strassenlaeufe/
url = "https://www.runnersworld.de/volkslaeufe-strassenlaeufe/"
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html)
table = soup.find('table')

links = []
for link in soup.findAll('a'):
    l = link.get('href')
    if l is None:
        continue
    if "laufkalender" in l:
        # clean
        if "laufeintragen" in l:
            continue
        if l == 'https://www.runnersworld.de/laufkalender/':
            continue
        links.append(l)    
links

['https://www.runnersworld.de/laufkalender/altstadtlaufweinheim/',
 'https://www.runnersworld.de/laufkalender/lissabonhalbmarathon/',
 'https://www.runnersworld.de/laufkalender/seemeilebremerhaven/',
 'https://www.runnersworld.de/laufkalender/badischemeilekarlsruhe/',
 'https://www.runnersworld.de/laufkalender/20-km-de-lausanne/',
 'https://www.runnersworld.de/laufkalender/neheimer-citylauf-arnsberg/',
 'https://www.runnersworld.de/laufkalender/wings-for-life-world-run-schweiz/',
 'https://www.runnersworld.de/laufkalender/saarbrueckener-city-lauf/',
 'https://www.runnersworld.de/laufkalender/lisbon-eco-marathon/',
 'https://www.runnersworld.de/laufkalender/wings-for-life-world-run-muenchen/',
 'https://www.runnersworld.de/laufkalender/heilbronnertrollingermarathon/',
 'https://www.runnersworld.de/laufkalender/duathlon-krailling/',
 'https://www.runnersworld.de/laufkalender/rahlstedterwandselaufhamburg/',
 'https://www.runnersworld.de/laufkalender/barcelona-marathon/',
 'https://www.run

In [84]:
# # remove everything after the 5th "/"
# import re
# pattern = ".+"
# test_string = 'https://www.runnersworld.de/laufkalender/salzburger-frauenlauf/'
# ls = test_string.split("/")
# clean_string = "/".join(ls[:5])
# clean_string    

'https://www.runnersworld.de/laufkalender/salzburger-frauenlauf'

In [96]:
# go through each link and extract information
volkslaeufe = {}
for url in links:
    info = scraper.get_result_similar(url)
    tbl = scrape_table(url)
    volkslaeufe[url] = [info, tbl]

In [177]:
# clean the extracted information
def remove_multiple_titles(autoscraper_result):
    """
    Sometimes the title of the run is captured 2 or 3 times in different variations
    # example 1:
        # 'Bitterfelder Goitzsche-Marathon',
        # 'Goitzsche-Marathon Bitterfeld',
        # ': Goitzsche-Marathon Bitterfeld',
    # example 2
        # 'Seemeile Bremerhaven',
        # ': Seemeile Bremerhaven',
    # example 3
        # 'Wings for Life World Run Schweiz',
        # ': Wings for Life World Run Schweiz',
    # example 4
        # 'Heilbronner Trollinger-Marathon',
        # ': Heilbronner Trollinger-Marathon'
    """
    x = autoscraper_result
    substrings = x[0].split()
    for sub in substrings:
        if sub in x[2]:
            x.pop(2)
            x.pop(1)
            break
        if sub in x[1]:
            x.pop(1)
            break
    return x

for key, val in volkslaeufe.items():
    autoscraper_result = val[0]
    if "StatischeSeiten" in autoscraper_result:
        autoscraper_result.remove("StatischeSeiten")
    volkslaeufe[key][0] = remove_multiple_titles(autoscraper_result)

In [184]:
i = np.random.randint(len(volkslaeufe))
volkslaeufe[list(volkslaeufe.keys())[i]]

[['Badische Meile Karlsruhe',
  'Stadtlauf / Citylauf',
  'Volkslauf / Straßenlauf',
  '08.05.2022',
  '76135 Karlsruhe, Hermann-Veit-Straße 3',
  'www.badischemeile.de',
  'info@badischemeile.de'],
 [['8,888 km ', '10.30 Uhr', '22,00 - 25,00 Euro']]]